In [1]:
from ultralytics import YOLO
import cv2
import cvzone
import math

# Load YOLO model
model = YOLO("C:/Users/49179/Desktop/Poker/runs/detect/train2/weights/best.pt")

# Class names
classNames = ['10C', '10D', '10H', '10S', '2C', '2D', '2H', '2S', '3C', '3D', '3H', '3S', '4C', '4D', '4H', '4S', '5C', '5D', '5H', '5S',
              '6C', '6D', '6H', '6S', '7C', '7D', '7H', '7S', '8C', '8D', '8H', '8S', '9C', '9D', '9H', '9S', 'AC', 'AD', 'AH', 'AS', 
              'JC', 'JD', 'JH', 'JS', 'KC', 'KD', 'KH', 'KS', 'QC', 'QD', 'QH', 'QS']

# Load image
image_path = "C:/Users/49179/Desktop/Poker/cards/test/images/001069757_jpg.rf.a1ae8dcf491f35edb5f5b2d629178ba2.jpg"
img = cv2.imread(image_path)

# Run YOLO model on the image
results = model.predict(img)

# Process the results
for r in results:
    boxes = r.boxes
    for box in boxes:
        # Bounding Box
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        w, h = x2 - x1, y2 - y1
        cvzone.cornerRect(img, (x1, y1, w, h))

        # Confidence
        conf = box.conf[0]
        conf = math.ceil((conf * 100)) / 100

        # Class Name
        cls = int(box.cls[0])
        label = f'{classNames[cls]} {conf}'
        cvzone.putTextRect(img, label, (max(0, x1), max(35, y1)), scale=1, thickness=1)

# Display the image with the detected boxes
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 640x640 1 5C, 1 KC, 2 KDs, 14.1ms
Speed: 4.0ms preprocess, 14.1ms inference, 75.6ms postprocess per image at shape (1, 3, 640, 640)



0: 288x512 2 4Ss, 1 6D, 1 7H, 1 KH, 1 QH, 24.0ms
Speed: 2.0ms preprocess, 24.0ms inference, 2.5ms postprocess per image at shape (1, 3, 288, 512)
['QH', '7H', '4S', '6D', 'KH']

0: 288x512 2 4Ss, 1 6D, 1 7H, 1 KH, 1 QH, 23.0ms
Speed: 3.0ms preprocess, 23.0ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 512)
['QH', '7H', '4S', '6D', 'KH']

0: 288x512 2 4Ss, 1 6D, 1 7H, 1 KH, 1 QH, 23.5ms
Speed: 0.0ms preprocess, 23.5ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 512)
['QH', '7H', '4S', '6D', 'KH']

0: 288x512 1 4S, 1 6D, 1 7H, 1 KH, 1 QH, 21.1ms
Speed: 2.0ms preprocess, 21.1ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 512)
['QH', '7H', '4S', '6D', 'KH']

0: 288x512 1 4S, 1 6D, 1 7H, 1 KH, 1 KS, 1 QH, 21.0ms
Speed: 1.0ms preprocess, 21.0ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 512)
['QH', '7H', '4S', 'KS', '6D', 'KH']

0: 288x512 1 4S, 1 6D, 1 7H, 1 KH, 1 KS, 1 QH, 20.0ms
Speed: 2.0ms preprocess, 20.0ms inference, 1.

In [99]:
def find_poker_hand(hand):
    ranks=[]
    suits = []
    possible_rank = []

    for card in hand:
        if len(card) == 2:
            rank=card[0]
            suit = card[1]
        else:
            rank= card[0:2]
            suit = card[2]

        if rank == "A":
            rank = 14
        elif rank == "J":
            rank = 11
        elif rank == "Q":
            rank = 12
        elif rank == "K":
            rank= 13
        ranks.append(int(rank))
        suits.append(suit)

    sorted_ranks = sorted(ranks)
    # check for royal_flush,straight_flush, flush and straight
    #first we check for flush
    if suits.count(suits[0]) == 5:
        if 10 in sorted_ranks and 11 in sorted_ranks and 12 in sorted_ranks and 13 in sorted_ranks and 14 in sorted_ranks:
            possible_rank.append("10")
        elif all(sorted_ranks[i] == sorted_ranks[i-1] +1 for i in range(1, len(sorted_ranks))):
            possible_rank.append("9")

        else:
            possible_rank.append("6")

    elif all(sorted_ranks[i]== sorted_ranks[i-1] + 1 for i in range(1, len(sorted_ranks))):
        possible_rank.append("5")


    #Check for Four of a kind, full_house, three of a kind
    values = list(set(sorted_ranks))

    if len(values) == 2:
        for i in values:
            if sorted_ranks.count(i) == 4:
                possible_rank.append("8")
            elif sorted_ranks.count(i) == 3:
                possible_rank.append("7")

    elif len(values) == 3:
        for i in values:
            if sorted_ranks.count(i) == 3:
                possible_rank.append("4")
            elif sorted_ranks.count(i) == 2:
                possible_rank.append("3")

    elif len(values) == 4:
        for i in values:
            if sorted_ranks.count(i) == 2:
                possible_rank.append("2")

    else:
        possible_rank.append("1")
    mapping = {11: "Jack", 12: "Queen", 13: "King", 14: "Ace"}
    Handranking = {"10": "Royal Flush", "9": "Straight Flush", "8": "Four of a Kind", "7": "Full House", "6": "Flush", "5": "Straight", "4": "Three of a Kind", "3": "Two pair", "2": "A Pair", "1": f"High Card: {mapping.get(max(sorted_ranks))}"} 
    
    Winning_Hand = Handranking[max(possible_rank)]
    return Winning_Hand


In [110]:
from ultralytics import YOLO
import cv2
import cvzone
import math

# Initialize webcam
cap = cv2.VideoCapture(0)  # Use 0 for built-in webcam, 1 for external webcam
cap.set(3, 1280)  # Set width
cap.set(4, 720)   # Set height

# Load YOLO model
model = YOLO("C:/Users/49179/Desktop/card_detection/runs/detect/train/weights/best.pt")

# Class names
classNames = ['10C', '10D', '10H', '10S', '2C', '2D', '2H', '2S', '3C', '3D', '3H', '3S', '4C', '4D', '4H', '4S', '5C', '5D', '5H', '5S',
              '6C', '6D', '6H', '6S', '7C', '7D', '7H', '7S', '8C', '8D', '8H', '8S', '9C', '9D', '9H', '9S', 'AC', 'AD', 'AH', 'AS', 
              'JC', 'JD', 'JH', 'JS', 'KC', 'KD', 'KH', 'KS', 'QC', 'QD', 'QH', 'QS']

while True:
    success, img = cap.read()  # Capture frame from webcam
    if not success:
        break

    img=cv2.flip(img,flipCode=1)
    # Run YOLO model on frame
    results = model.predict(img, stream=True)
    hand = []

    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            w, h = x2 - x1, y2 - y1
            cvzone.cornerRect(img, (x1, y1, w, h))
            
            # Confidence
            conf = box.conf[0]
            conf = math.ceil((conf * 100)) / 100
            
            # Class Name
            cls = int(box.cls[0])
            label = f'{classNames[cls]} {conf}'
            cvzone.putTextRect(img, label, (max(0, x1), max(35, y1)), scale=1, thickness=1)

            # Add to hand if confidence is high enough
            if conf > 0.5:
                hand.append(classNames[cls])



    # Uncomment the lines below if you have a function `PokerHandFunction.findPokerHand`
    # that identifies the type of poker hand based on detected cards.
    '''
    hand = list(set(hand))
    if len(hand) == 5:
        results = PokerHandFunction.findPokerHand(hand)
        cvzone.putTextRect(img, f'Your Hand: {results}', (300, 75), scale=3, thickness=5)
    '''
    hand = list(set(hand))
    print(hand)
    if len(hand) == 5:
        results = find_poker_hand(hand)
        cvzone.putTextRect(img, f'Your Hand: {results}', (300, 75), scale=3, thickness=5)
        print(results)
    # Display the image
    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()



0: 288x512 2 10Hs, 1 4H, 1 AH, 2 JHs, 1 KH, 2 QHs, 22.5ms
Speed: 1.0ms preprocess, 22.5ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 512)
['QH', 'AH', '4H', 'JH', '10H', 'KH']

0: 288x512 2 10Hs, 1 AH, 2 JHs, 2 KHs, 2 QHs, 21.5ms
Speed: 2.0ms preprocess, 21.5ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 512)
['QH', 'AH', 'JH', '10H', 'KH']
Royal Flush

0: 288x512 2 10Hs, 1 AH, 2 JHs, 2 KHs, 2 QHs, 18.0ms
Speed: 2.0ms preprocess, 18.0ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 512)
['QH', 'AH', 'JH', '10H', 'KH']
Royal Flush

0: 288x512 2 10Hs, 1 4H, 1 7H, 1 AH, 2 JHs, 1 KH, 2 QHs, 19.0ms
Speed: 1.5ms preprocess, 19.0ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 512)
['QH', '4H', 'JH', '10H', 'KH']
Flush

0: 288x512 2 10Hs, 1 4H, 1 7H, 1 AH, 2 JHs, 1 KH, 2 QHs, 19.0ms
Speed: 2.0ms preprocess, 19.0ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 512)
['QH', '4H', 'JH', '10H', 'KH']
Flush

0: 288x512 2 10